In [ ]:
import country_converter as coco
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt

cc = coco.CountryConverter()

# Téléchargement des données mondiales de covid

In [ ]:
URL_DATASET = r'https://raw.githubusercontent.com/datasets/covid-19/master/data/countries-aggregated.csv'
df1 = pd.read_csv(URL_DATASET)

In [ ]:
df1.head()

# Coversion des noms de pays vers leur code ISO3 (3 lettres)

In [ ]:
iso3_codes = coco.convert(names=list(df1.Country.unique()), to='ISO3')
df2 = pd.DataFrame({"Country": df1.Country.unique(), "iso3": iso3_codes})
df1_country_codes = df1.merge(df2, on="Country")

df1_country_codes.head()

# Prétraitement 

Il faut effectuer des moyennes glissantes / resampler les données sur une semaine et par pays pour avoir des informations pertinentes. On en profite pour rajouter les informations de code ISO3 des pays. 

In [ ]:
df_resampled = []

for country, group in df1.groupby("Country"):
    current = group.set_index(pd.to_datetime(group.Date)).rolling("7d").mean()
    current["Country"] = country
    current["iso3"] = coco.convert(names=[country], to='ISO3')
    df_resampled.append(current)

df_resampled = pd.concat(df_resampled)

# Réalisation de graphiques temporels

In [ ]:
ax = df_resampled[df_resampled.Country == "France"].Confirmed.plot()
ax.set_title("Nombre de cas cumulés en France");
plt.savefig("covid_cas_cumules.png")

In [ ]:
ax = df_resampled[df_resampled.Country == "France"].Confirmed.diff().plot()
ax.set_title("Nombre de cas par jours en France");
plt.savefig("covid_cas_journaliers.png")

# Réalisation d'une carte en mode *cloropleth*

Pour générer une carte avec l'intéreur des pays coloré selon le nombre de contaminations on utilis plotly et mapbox.

In [ ]:
fig = px.choropleth(data_frame = df_resampled,
                locations= "iso3",
                color= "Confirmed",  # value in column 'Confirmed' determines color
                hover_name= "Country",
                color_continuous_scale= 'RdYlGn',  #  color scale red, yellow green
                animation_frame=df_resampled.index.astype(str))
fig.show()

# Réalisation de scatter map

Pour afficher des ronds sur les pays il faut connaitre la position des barycentres des pays. 

Le fichier CSV possède des problèmes de formatage, il faut donc supprimer les guillemets présents en trop. Et modifier le type des colonnes.

In [ ]:
#countries_lat_lon = pd.read_csv("http://worldmap.harvard.edu/download/wfs/34645/csv?outputFormat=csv&service=WFS&request=GetFeature&format_options=charset%3AUTF-8&typename=geonode%3Acountry_centroids_az8&version=1.0.0")
countries_lat_lon = pd.read_csv("https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv", header=0)

In [ ]:
countries_lat_lon = countries_lat_lon.apply(lambda x: x.str.replace('"', '').str.strip())
countries_lat_lon["Latitude (average)"] = countries_lat_lon["Latitude (average)"].astype(np.float)
countries_lat_lon["Longitude (average)"] = countries_lat_lon["Longitude (average)"].astype(np.float)

countries_lat_lon.info()

Maintenant, nous pouvons merger les deux datasets entre eux pour récupérer les informations qui nous intéressent (les cas de covid + latitude / longitude)

In [ ]:
resampled_lat_lon = df_resampled.\
    reset_index().\
    merge(
    countries_lat_lon[["Alpha-3 code", "Latitude (average)", "Longitude (average)"]], 
    left_on="iso3", 
    right_on="Alpha-3 code", 
    how='right').\
    set_index("Date").\
    dropna()

resampled_lat_lon.head()

In [ ]:
fig = px.scatter_mapbox(resampled_lat_lon,
                lat="Latitude (average)",  
                lon="Longitude (average)",
                color="Confirmed", size="Confirmed",
                color_continuous_scale=px.colors.sequential.Aggrnyl, 
                size_max=70, zoom=0.75, hover_name='Country', 
                hover_data = ['Confirmed', 'Deaths', 'Recovered'], 
                title = 'Visualizing spread of COVID from 22/1/2020', 
                animation_frame=resampled_lat_lon.index.astype(str)

)
fig.update_mapboxes(style='open-street-map')
fig.show()